In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
## Load data 
data = pd.read_csv(r"C:\Users\Kunal Rana\Downloads\DataScienceCleanStatement.csv")

## Overview of dataset
data.head()

,Date,Description,InternationTransactionAmount,Category,Amount,TransactionUserName,TransactionType,IsoCurrency,Locality,City,State,Country,Unnamed: 12
0,12/12/2019,THE ROOM GURGAON ...,0,Video game,-127,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,NaN
1,12/12/2019,CUT AND STYLEGURGAON ...,0,Occupation,-499,JASJIT KAUR,Domestic,INR,Sector 4,Gurugram,Haryana,India,NaN
2,12/12/2019,MUMUSOGURGAON ...,0,,-388,JASJIT KAUR,Domestic,INR,Sector 49,Gurugram,Haryana,India,NaN
3,12/12/2019,THE ROOM GURGAON ...,0,Video game,-108,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,NaN
4,12/12/2019,THE ROOM GURGAON ...,0,Video game,-50,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 13 columns):
 #   Column                                                               Non-Null Count  Dtype 
---  ------                                                               --------------  ----- 
 0   Date                                                                 1470 non-null   object
 1   Description                                                          1470 non-null   object
 2   InternationTransactionAmount                                         1470 non-null   object
 3     Category                                                           1470 non-null   object
 4     Amount                                                             1470 non-null   object
 5   TransactionUserName                                                  1470 non-null   object
 6    TransactionType                                                     1470 non-null   object
 7   IsoCurrency    

In [6]:
data.describe()

,Date,Description,InternationTransactionAmount,Category,Amount,TransactionUserName,TransactionType,IsoCurrency,Locality,City,State,Country,Unnamed: 12
count,1470,1470,1470,1470,1470,1470,1470,1470,1302,1270,1245,1307,4
unique,408,495,35,153,991,7,4,5,186,41,21,7,1
top,1/8/2019,THE ROOM GURGAON ...,0,,-50,JASJIT KAUR,Domestic,INR,Central Park II,Gurugram,Haryana,India,India
freq,13,122,1436,508,25,755,1433,1433,227,930,935,1260,4


In [7]:
data.columns

Index(['Date      ',
       'Description                                                        ',
       'InternationTransactionAmount', '  Category', '  Amount',
       'TransactionUserName ', ' TransactionType', 'IsoCurrency', 'Locality',
       'City', 'State', 'Country', 'Unnamed: 12'],
      dtype='object')

In [8]:
## Make a copy of dataset 
data_copy = data.copy()

In [9]:
data_copy.shape

(1470, 13)

In [10]:
## Drop Unamed column coz it is irrelevant
data_copy.drop(['Unnamed: 12'], axis = 1, inplace=True)

In [11]:
## Remove irrelevant spaces from column name
data_copy.columns = data_copy.columns.str.replace(' ', '')
data_copy.columns

Index(['Date', 'Description', 'InternationTransactionAmount', 'Category',
       'Amount', 'TransactionUserName', 'TransactionType', 'IsoCurrency',
       'Locality', 'City', 'State', 'Country'],
      dtype='object')

In [12]:
## Locality is subset of City, City < State & State is subset of Country.
## Check % of values in country is missing.
data_copy['Country'].isnull().sum() * 100 / len(data_copy)

11.08843537414966

In [13]:
## Fill missing values in COuntry column to most frequent value in dataset.
data_copy.Country.mode()
data_copy['Country'].fillna(data_copy['Country'].mode()[0], inplace = True)
data_copy['Country'].isnull().sum()

0

In [14]:
## Inspect vlaues in country and pre-process it.
data_copy['Country'].value_counts().index

## Replace state name with country name in COuntry column
data_copy["Country"].replace({"Haryana": "India", "Delhi": "India", "Rajasthan":"India"}, inplace=True)

In [15]:
## Analyse and clean InternationTransactionAmount column in dataset
data_copy['InternationTransactionAmount'] = data_copy.InternationTransactionAmount.str.replace(' ', '')
data_copy.InternationTransactionAmount.unique()

array(['0', '36', 'F-2/7OKHDelhi', '669.74', '31.83', '183.95', '63.56',
       '1.95', '105', '84.7', '5', '22.8', '778.74', '214.4', '14.4',
       '60.89', '65.33', '19.17', '238.73', '12.3', '489.63', '61.77',
       '34.4', '209', '42', '274', '8', '99.9', '80.8', '21.2', '95',
       '5.5', '13', '227', '368'], dtype=object)

In [16]:
## Remove unnecessary value in InternationTransactionAmount column
data_copy.drop(data_copy.index[data_copy['InternationTransactionAmount'] == 'F-2/7OKHDelhi'], inplace = True)

In [17]:
## Analyse and clean State column  
data_copy.State.unique()
data_copy["State"].replace({"हरियाणा": "Haryana","Jaipur":"Rajasthan","Gurugram": "Haryana"}, inplace=True)

## Fill missing vlaue from most frequent vlaue in column
data_copy['State'].fillna(data_copy['State'].mode()[0], inplace = True)

In [18]:
## Analyse and clean City column  
data_copy.City.unique()

## Change City name in string only, however it consist country name but it is not efficient to find particular city.
data_copy["City"].replace({"गुरुग्राम": "Gurugram"}, inplace=True)

## Fill missing vlaue from most frequent vlaue in column
data_copy['City'].fillna(data_copy['City'].mode()[0], inplace = True)

In [19]:
## Analyse and clean Locality column  
data_copy.Locality.unique()

## Fill missing vlaue from most frequent vlaue in column
data_copy['Locality'].fillna(data_copy['Locality'].mode()[0], inplace = True)

In [20]:
## Now, check null values in dataset
data_copy.isnull().sum()

Date                            0
Description                     0
InternationTransactionAmount    0
Category                        0
Amount                          0
TransactionUserName             0
TransactionType                 0
IsoCurrency                     0
Locality                        0
City                            0
State                           0
Country                         0
dtype: int64

In [21]:
## Clean Description column i.e remove spacing 
data_copy['Description'] = data_copy.Description.str.replace(' ', '')
data_copy.Description.unique()[:10]

array(['THEROOMGURGAON', 'CUTANDSTYLEGURGAON', 'MUMUSOGURGAON',
       'JAIPURSPACEJAIPUR', 'CAFECOFFEEDAYHIGHWAYKI',
       'CAFECOFFEEDAYALANKARPL', 'LIFESTYLEINTERNATIONAJAIPUR',
       'NEWU47JAIPUR', 'COCONUTRESTAURANTJAIPUR', 'UCBJAIPUR'],
      dtype=object)

In [22]:
## Clean Amount column i.e remove spacing 
data_copy['Category'] = data_copy.Category.str.replace(' ', '')
data_copy.Category.unique()[-15:]

array(['Technologycompany', 'SongbyRhye', 'King', 'SongbyTheBotherers',
       'Indianteacher', 'Guru', 'Televisioncompany', 'Agency',
       'Medication', '2001film', 'Indianactor', 'Supermarketcompany',
       'NovelbyLewisCarroll', 'Chemicalcompound', 'CityinJapan'],
      dtype=object)

In [23]:
## Clean and remove extra spacing in ISCurrency column 
data_copy['IsoCurrency'] = data_copy.IsoCurrency.str.replace(' ', '')
data_copy.IsoCurrency.unique()

array(['INR', 'USD', 'SGD', 'MYR', 'Domestic'], dtype=object)

In [24]:
## Clean and remove extra spacing in TransactionType column 
data_copy['TransactionType'] = data_copy.TransactionType.str.replace(' ', '')

data_copy.TransactionType.mode()
## Transaction Type column contains name rather than type, need to change it with most frequent value i.e Domestic.

data_copy["TransactionType"].replace({"JASJITKAUR": "Domestic", "VIBHOREGOYAL": "Domestic" }, inplace=True)

data_copy.TransactionType.unique()

array(['Domestic', 'International'], dtype=object)

In [25]:
## Clean and remove extra spacing in TransactionUserName column 
data_copy.TransactionUserName.mode()

## Change random numbers in UserName column to most frequent value.
data_copy["TransactionUserName"].replace({"-2899": "JASJITKAUR", "-2348": "JASJITKAUR","-2391": "JASJITKAUR"}, inplace=True)

## remove extra spacing in TransactionName column 
data_copy['TransactionUserName'] = data_copy.TransactionType.str.replace(' ', '')

data_copy.TransactionUserName.unique()

array(['Domestic', 'International'], dtype=object)

In [26]:
## Now, change numerical data containing column to numerical data type
data_copy["Amount"] = pd.to_numeric(data_copy["Amount"], errors='coerce')

In [27]:
data_copy["InternationTransactionAmount"] = pd.to_numeric(data_copy["InternationTransactionAmount"], errors='coerce')

In [28]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1469 entries, 0 to 1469
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date                          1469 non-null   object 
 1   Description                   1469 non-null   object 
 2   InternationTransactionAmount  1469 non-null   float64
 3   Category                      1469 non-null   object 
 4   Amount                        1466 non-null   float64
 5   TransactionUserName           1469 non-null   object 
 6   TransactionType               1469 non-null   object 
 7   IsoCurrency                   1469 non-null   object 
 8   Locality                      1469 non-null   object 
 9   City                          1469 non-null   object 
 10  State                         1469 non-null   object 
 11  Country                       1469 non-null   object 
dtypes: float64(2), object(10)
memory usage: 149.2+ KB


In [29]:
## Extract month and year from Date column of DD/MM/YYYY format.
data_copy['year'] = pd.DatetimeIndex(data_copy['Date']).year

In [30]:
## Extract month
data_copy['month'] = pd.DatetimeIndex(data_copy['Date']).month

In [31]:
## Extract day also from Date
data_copy['day'] = pd.DatetimeIndex(data_copy['Date']).day

In [32]:
data_copy.tail()

,Date,Description,InternationTransactionAmount,Category,Amount,TransactionUserName,TransactionType,IsoCurrency,Locality,City,State,Country,year,month,day
1465,6/7/2019,KFCSEC47SOHNAROADGGURGAON,0.0,Fastfoodrestaurantcompany,-120.0,Domestic,Domestic,INR,D1 Block,Gurugram,Haryana,India,2019,6,7
1466,7/7/2019,SPICEJETLIMITEDGURGAON,0.0,Airline,-1798.0,Domestic,Domestic,INR,Phase III,Gurugram,Haryana,India,2019,7,7
1467,7/7/2019,CAFECOFFEEDAYILDTRADE,0.0,Cafecompany,-298.0,Domestic,Domestic,INR,Sohna Road,Gurugram,Haryana,India,2019,7,7
1468,7/7/2019,SHOPPERSSTOPLTDGURGAON,0.0,Departmentstorecompany,-1852.0,Domestic,Domestic,INR,Heritage City,Gurugram,Haryana,India,2019,7,7
1469,11/7/2019,TATAINDICOM-DOCOMOMUMBAI,0.0,FormerPresidentofSouthAfrica,-1297.0,Domestic,Domestic,INR,Mahim,Mumbai,Maharashtra,India,2019,11,7


In [33]:
## Finally, save the dataset in new csv file.
data_copy.to_csv('Clean_Data_Science.csv')